In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.layers.core import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pathlib
from tensorflow.keras.optimizers import SGD

In [ ]:
img_size = [224,224]

train_path = "/content/drive/MyDrive/covid_dataset/train"
test_path = "/content/drive/MyDrive/covid_dataset/test"
val_path = "/content/drive/MyDrive/covid_dataset/val"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
)
                                   
                                   

test_datagen = ImageDataGenerator(rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_set = train_datagen.flow_from_directory(train_path,
                                              target_size = (224,224),
                                              batch_size = 32,
                                              
                                              class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = 'categorical')
val_set = test_datagen.flow_from_directory(val_path,
                                           target_size = (224,224),
                                           batch_size = 32,
                                           class_mode = 'categorical')
                                            

Found 20218 images belonging to 2 classes.
Found 2118 images belonging to 2 classes.
Found 4232 images belonging to 2 classes.


In [ ]:
len(train_set)

In [ ]:


resnet_model= tf.keras.applications.ResNet50(include_top=False,
                                                 input_shape=(224,224,3),
                                                 pooling='avg',
                                                 classes=2,
                                                 weights='imagenet')
n = len(resnet_model.layers)


for layer in resnet_model.layers[-2:]:
  layer.trainable = True
  print(layer)

for layer in resnet_model.layers[:-2]:
  layer.trainable = False


94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
model = tf.keras.Sequential([
    resnet_model,
    layers.Flatten(),
    layers.Dense(1024, activation = 'relu'),
    layers.Dense(2, activation = 'softmax')
])

In [ ]:
model.summary()

In [ ]:

model.compile(optimizer =  Adam(learning_rate=0.0008,),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
              metrics = ['accuracy'])

In [ ]:
initial_epochs=  10
training = model.fit(train_set,
                     epochs = initial_epochs,
                     validation_data = val_set)

Epoch 1/10
632/632 [==============================] - 10054s 16s/step - loss: 0.5932 - accuracy: 0.6930 - val_loss: 0.4709 - val_accuracy: 0.8277
Epoch 2/10
632/632 [==============================] - 117s 185ms/step - loss: 0.5049 - accuracy: 0.7625 - val_loss: 0.4171 - val_accuracy: 0.8308
Epoch 3/10
632/632 [==============================] - 114s 181ms/step - loss: 0.4223 - accuracy: 0.8137 - val_loss: 0.4320 - val_accuracy: 0.8140
Epoch 4/10
632/632 [==============================] - 114s 180ms/step - loss: 0.3842 - accuracy: 0.8316 - val_loss: 0.5779 - val_accuracy: 0.6697
Epoch 5/10
632/632 [==============================] - 114s 181ms/step - loss: 0.3792 - accuracy: 0.8332 - val_loss: 0.4325 - val_accuracy: 0.8313
Epoch 6/10
632/632 [==============================] - 114s 181ms/step - loss: 0.3582 - accuracy: 0.8436 - val_loss: 0.3784 - val_accuracy: 0.8405
Epoch 7/10
632/632 [==============================] - 116s 183ms/step - loss: 0.3636 - accuracy: 0.8412 - val_loss: 0.3724 -

In [ ]:
model.evaluate(test_set, verbose = 2)

67/67 - 428s - loss: 0.7003 - accuracy: 0.8107 - 428s/epoch - 6s/step


[0.700348973274231, 0.8106704354286194]